In [ ]:
from dotenv import load_dotenv
import os
from langchain.agents import tool
from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description
from langchain_openai import ChatOpenAI
from langchain.schema import AgentAction, AgentFinish

In [ ]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
@tool
def get_text_length(text: str) -> int:
    """return the length of a text"""
    return len(text.strip("\n").strip('"').strip("'"))

In [ ]:
tools = [get_text_length]

In [ ]:
template="""
Answer the following questions as best you can. You have access to the following tools: {tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
Thought: {agent_scratchpad}
"""

In [ ]:
prompt = PromptTemplate.from_template(template=template).partial(tools=render_text_description(tools), tool_names=", ".join([t.name for t in tools]))

In [ ]:
llm = ChatOpenAI(
  model="gpt-3.5-turbo",
  temperature=0,
  api_key=OPENAI_API_KEY,
  model_kwargs={"stop": ["\nObservation", "Observation"] }
)

In [ ]:
agent_scratchpad = []

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["agent_scratchpad"]),
    }
    | prompt
    | llm
    | ReActSingleInputOutputParser()
)

In [ ]:
def find_tool(tools, name):
    for tool in tools:
        if tool.name == name:
            return tool
    return None

In [ ]:
step = None
while not isinstance(step, AgentFinish):
    step = agent.invoke(
        {
            "input": "What is the length of the word: pistacho",
            "agent_scratchpad": agent_scratchpad,
        }
    )
    print(step)
    if isinstance(step, AgentAction):
        tool_name = step.tool
        tool = find_tool(tools, step.tool)
        observation = tool.func(str(step.tool_input))
        print(f"{observation=}")
        agent_scratchpad.append((step, str(observation)))


In [ ]:
if isinstance(step, AgentFinish):
    print(step.return_values)